In [1]:
import numpy as np
import os
import torch
import config
import cv2
import sys

from model_structures.UNET_3d_2l import UNet3D
from tqdm import tqdm
from copy import deepcopy as copy

# Load the PyTorch model
path_model = "./saved_models/LSM/model_for_vasc_3d_2l_9071617.pth"
name = path_model.split("/")[-2]
model = UNet3D()
model.load_state_dict(torch.load(path_model, map_location="cpu"))

# Set the model to evaluation mode
model.eval()

# Directory paths for input and output data
# input_dir = "./preparations/data/indata"
# output_dir = "./preparations/data/outdata"

# Get a list of image files in the input directory
# image_files = os.listdir(input_dir)

def get_prediction(img, margin):
    if margin != 0:
        raise ValueError
    with torch.no_grad():
        input_tensor = torch.from_numpy(img).unsqueeze(0).float()
        output = model(input_tensor)
    output_np = output.numpy()
    return output_np

def calculate_binary_array(count_array, total_array):
    # Avoid division by zero by setting a mask for total_array equal to zero
    mask = total_array == 0
    
    # Initialize binary_array with all zeros
    new_array = np.zeros_like(count_array)
    
    # Calculate binary_array where total_array is not zero
    new_array[~mask] = (count_array[~mask] / total_array[~mask]) >= 0.5
    
    return new_array

arg = 1
# Create a nrrd file from a prediction based on the input directory
image_array = np.load("./unprocessed_data/LSM/volume_input.npy")
if len(sys.argv) > 1 or arg > 0:
    if sys.argv[1] == "0" or arg == 1:
        blur_array = [cv2.GaussianBlur(image_array[i], (5, 5), 0) for i in range(image_array.shape[0])]
        image_init = np.expand_dims(blur_array, axis=0)
    else:
        # Apply Pseudo flat field correction
        blur_array = [cv2.GaussianBlur(image_array[i], (127, 127), 0) for i in range(image_array.shape[0])]

        image_pff_array = [cv2.divide(image_array[i], blur_array[i], scale=255) for i in range(image_array.shape[0])]

        # normalize
        image_pff_array = (image_pff_array - np.min(image_pff_array)) / (np.max(image_pff_array) - np.min(image_pff_array))

        image_init = np.expand_dims(image_pff_array, axis=0)
else:
    image_init = np.expand_dims(image_array, axis=0)

# Print the shape of the input and output arrays
shape = image_init.shape
print("Initial shape:", shape)

# Parameters
step_ratio = 2
step = int(config.HEIGHT / step_ratio)
size = config.HEIGHT
print(step, size)
margin = 0

# Create 0 copies of inputs
count_array = np.zeros_like(image_init)
total_array = np.zeros_like(image_init)

index = 0
if not os.path.exists("./nrrd"):
    os.makedirs("./nrrd")

# amount
i_amount = np.ceil(shape[1] / config.NUM_PICS)
j_amount = np.ceil(shape[2] / step)
k_amount = np.ceil(shape[3] / step)

for i in tqdm(range(0, int(i_amount)), "Processing"):
    for j in range(0, int(j_amount)):
        for k in range(0, int(k_amount)):
            # a b c based on i j k
            slice_tmp = image_init[:,
                                   min(i * config.NUM_PICS, shape[1] - config.NUM_PICS) : min((i + 1) * config.NUM_PICS, shape[1]),
                                   min(j * step, shape[2] - config.HEIGHT) : min(j * step + config.HEIGHT, shape[2]),
                                   min(k * step, shape[3] - config.WIDTH) : min(k * step + config.WIDTH, shape[3])]


            array_tmp = get_prediction(slice_tmp, margin)[0]
            # cv2.imshow(f"img", cv2.resize(slice_tmp[0][0], (512, 512)))
            # cv2.imshow("pred", cv2.resize(array_tmp[0][0], (512, 512)))
            # cv2.waitKey(0)

            count_array[:,
                        min(i * config.NUM_PICS, shape[1] - config.NUM_PICS) : min((i + 1) * config.NUM_PICS, shape[1]),
                        min(j * step, shape[2] - config.HEIGHT) : min(j * step + config.HEIGHT, shape[2]),
                        min(k * step, shape[3] - config.WIDTH) : min(k * step + config.WIDTH, shape[3])] = array_tmp
            total_array[:,
                        min(i * config.NUM_PICS, shape[1] - config.NUM_PICS) : min((i + 1) * config.NUM_PICS, shape[1]),
                        min(j * step, shape[2] - config.HEIGHT) : min(j * step + config.HEIGHT, shape[2]),
                        min(k * step, shape[3] - config.WIDTH) : min(k * step + config.WIDTH, shape[3])] = 1

binary_array = calculate_binary_array(count_array, total_array)[0] 
# Save the binary array as a .npy file
if not os.path.exists("./processed_npy"):
    os.makedirs("./processed_npy")

np.save(f"./processed_npy/{name}.npy", binary_array)

Initial shape: (1, 200, 200, 200)
64 128


Processing: 100%|██████████| 13/13 [01:18<00:00,  6.03s/it]


In [12]:
import cv2
ind = 12
print(total_array.shape)
norm = (total_array[0][ind] > - np.min(total_array[0][ind])) / (np.max(total_array[0][ind]) - np.min(total_array[0][ind]))
cv2.imshow("total", cv2.resize(((norm * 255).astype(np.uint8)).astype(np.uint8), (512, 512)) * 255)
cv2.imshow("image", cv2.resize(image_array[ind], (512, 512)))
cv2.imshow("prediction", cv2.resize(binary_array[ind], (512, 512)))

cv2.waitKey(0)
cv2.destroyAllWindows()
print(np.min(total_array), np.max(total_array))

(1, 400, 400, 1000)


C:\Users\atashitk\AppData\Local\Temp\ipykernel_11540\1908260342.py:4: RuntimeWarning: divide by zero encountered in divide
  norm = (total_array[0][ind] > - np.min(total_array[0][ind])) / (np.max(total_array[0][ind]) - np.min(total_array[0][ind]))
C:\Users\atashitk\AppData\Local\Temp\ipykernel_11540\1908260342.py:5: RuntimeWarning: invalid value encountered in cast
  cv2.imshow("total", cv2.resize(((norm * 255).astype(np.uint8)).astype(np.uint8), (512, 512)) * 255)


1.0 1.0


In [13]:
# Save bnary_array as nnrd
import nrrd
name = path_model.split("/")[-2]
# create a folder
if not os.path.exists(f"./nrrd/{name}"):
    os.makedirs(f"./nrrd/{name}")

nrrd.write(f"./nrrd/{name}/pred_orig.nrrd", binary_array)

flipped = np.flip(np.moveaxis(np.flip(binary_array, axis=2), [0, -2], [2, 1]), axis=0)
nrrd.write(f"./nrrd/{name}/pred_flipped.nrrd", flipped)

# original data
nrrd.write(f"./nrrd/{name}/images_orig.nrrd", image_array)

flipped = np.flip(np.moveaxis(np.flip(image_array, axis=2), [0, -2], [2, 1]), axis=0)
nrrd.write(f"./nrrd/{name}/images_flipped.nrrd", flipped)

In [4]:
# Import 2 nrrd files and apply "OR" on the two files
import numpy as np
import nrrd

nrrd1 = nrrd.read(f"./nrrd/micro_ct_v2/flipped.nrrd")[0]
nrrd2 = nrrd.read(f"./nrrd/micro_ct_v3/flipped.nrrd")[0]

nrrd_array = 1 - (1 - nrrd1[0]) * (1 - nrrd2 // 255)
nrrd.write(f"./nrrd/v2_or_v3.nrrd", nrrd_array)


In [29]:
nrrd_array = np.logical_or(nrrd1[0], nrrd2 / 255)
nrrd.write(f"./nrrd/v2_or_v3.nrrd", nrrd_array)

NameError: name 'nrrd1' is not defined